In [1]:
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, jaccard_score, f1_score, precision_score, recall_score
from skimage.metrics import structural_similarity as ssim
from sklearn import preprocessing
from keras.utils import to_categorical
from keras import regularizers
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.layers import Input, Dense, Dropout, Lambda, GlobalAveragePooling2D
from keras.src.legacy.preprocessing.image import ImageDataGenerator

import random
import re
import glob
from tqdm import tqdm
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import time
import gc

2025-04-07 14:16:47.692629: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-07 14:16:47.761944: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-07 14:16:47.792338: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-07 14:16:48.518268: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"GPUs disponíveis: {gpus}")
else:
    print("Nenhuma GPU encontrada.")

GPUs disponíveis: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1744046224.857093    1891 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744046226.671257    1891 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744046226.671402    1891 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.


In [3]:
# nome = 'PACS'
nome = 'Yildirim'
#nome = 'Todos'

imgs_path = 'Bases/Dataset' + nome

output_dir = "13-CNN-espectro-ResNet152 results (2%)/"
os.makedirs(output_dir, exist_ok=True)

TEST_SIZE = 0.1
VALIDATION_SIZE = 0.1

SIMILARITY = 0.75

EPOCHS = 200
BATCH_SIZE = 32

HIGH_FREQ_THRESHOLD = 2

In [4]:
def get_next_filename(output_folder, base_name, type):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    image_index = 0
    
    while True:
        output_filename = f"{base_name}_{image_index}.{type}"
        output_path = os.path.join(output_folder, output_filename)
        
        if not os.path.exists(output_path):
            return output_filename
            
        image_index += 1

In [5]:
def extract_number(filename):
    match = re.search(r'(\d+)', filename)
    return int(match.group(0)) if match else 0

In [6]:
def sort_files_numerically(file_paths):
    return sorted(file_paths, key=lambda x: extract_number(x))

In [7]:
def read_dataset(path, jpg, png):
    print(f'Reading dataset...\n')
    
    img_type = []
    images = []
    image_paths = []

    if jpg:
        img_type.append('*.jpg')
    if png:
        img_type.append('*.png')

# Normal

    normal_path = os.path.join(path, 'Train', 'Normal')
    print(f'Reading Normal images from: {normal_path}')
    
    for img_type_pattern in img_type:
        img_paths = glob.glob(os.path.join(normal_path, img_type_pattern))
        img_paths = sort_files_numerically(img_paths)
        
        for img_path in img_paths:
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (224, 224))
            images.append(img)
            image_paths.append(img_path)

# Kidney

    kidney_stone_path = os.path.join(path, 'Train', 'Kidney_stone')
    print(f'Reading Kidney_stone images from: {kidney_stone_path}')
    
    for img_type_pattern in img_type:
        img_paths = glob.glob(os.path.join(kidney_stone_path, img_type_pattern))
        img_paths = sort_files_numerically(img_paths)
        
        for img_path in img_paths:
            img = cv2.imread(img_path, 0)
            img = cv2.resize(img, (224, 224))
            images.append(img)
            image_paths.append(img_path)

# ...
    
    images = np.array(images)
    image_paths = np.array(image_paths)

    return images, image_paths

In [8]:
all_X, all_image_paths = read_dataset(path=imgs_path, jpg=True, png=True)

Reading dataset...

Reading Normal images from: Bases/DatasetYildirim/Train/Normal
Reading Kidney_stone images from: Bases/DatasetYildirim/Train/Kidney_stone


In [9]:
def freq_spec(image, threshold, add_noise):
    f = np.fft.fft2(image)
    fshift = np.fft.fftshift(f)

    if add_noise:
        amplification_factor = 0.5
        rows, cols = image.shape
        crow, ccol = rows//2, cols//2

        corner = np.random.randint(0, 4)
        
        mask = np.zeros((rows, cols), dtype=bool)
        
        if corner == 0:    # Canto superior esquerdo
            mask[:crow - threshold, :ccol - threshold] = True
        elif corner == 1:  # Canto superior direito
            mask[:crow - threshold, ccol + threshold:] = True
        elif corner == 2:  # Canto inferior esquerdo
            mask[crow + threshold:, :ccol - threshold] = True
        else:              # Canto inferior direito
            mask[crow + threshold:, ccol + threshold:] = True

        fshift[mask] *= amplification_factor

    magnitude_spectrum_high = 20 * np.log(np.abs(fshift) + 1)

    return magnitude_spectrum_high

In [10]:
def save_specs(images, threshold, add_noise):
    specs = []
    for img in images:
        specs.append(freq_spec(img, threshold, add_noise))
    return specs

In [11]:
all_specs = []
all_labels = []

half = len(all_X) // 2
for i, img in enumerate(all_X):
    if i < half:
        all_specs.append(freq_spec(img, 100 - HIGH_FREQ_THRESHOLD, add_noise=False))
        all_labels.append(0)
    else:
        all_specs.append(freq_spec(img, 100 - HIGH_FREQ_THRESHOLD, add_noise=True))
        all_labels.append(1)

all_specs = np.array(all_specs)
all_labels = np.array(all_labels)

In [12]:
print("Quantidade das imagens:", all_specs.shape)
print("Exemplo dos labels (False = original, True = com ruído):", all_labels)

Quantidade das imagens: (1799, 224, 224)
Exemplo dos labels (False = original, True = com ruído): [0 0 0 ... 1 1 1]


In [13]:
X_train, X_test, y_train, y_test, paths_train, paths_test = train_test_split(
        all_specs, all_labels, all_image_paths,
        test_size=TEST_SIZE, 
        stratify=all_labels,
#       random_state=53
    )

In [14]:
def plot_specs(specs, title):
    total_images = len(specs)
    imgs_per_figure = 100
    cols = 5
    rows = (imgs_per_figure + cols - 1) // cols

    for start in range(0, total_images, imgs_per_figure):
        plt.figure(figsize=(cols * 3, rows * 3))
        end = min(start + imgs_per_figure, total_images)
        
        for i in range(start, end):
            plt.subplot(rows, cols, i - start + 1)
            plt.imshow(specs[i], cmap='gray')
            plt.title(f"{title} - freq spec: img {i+1}")
            plt.axis('off')
            
        plt.tight_layout()
        plt.show()

In [15]:
# Plotar todos os espectros de teste
#plot_specs(X_test, title="test")

In [16]:
#pausa de teste

In [17]:
# Defines the model layers (now for ResNet50)
def model_resnet152():  # (Optional: Rename the function for clarity)
    # Camada de entrada para imagens em escala de cinza (mantido igual)
    inputs = Input(shape=(224, 224, 1))
    
    # 1. Conversão para 3 canais
    x = Lambda(
        lambda x: tf.stack([x[..., 0]]*3, axis=-1),
        output_shape=(224, 224, 3)
    )(inputs)
    
    # 2. Pré-processamento ResNet
    x = tf.keras.applications.resnet.preprocess_input(x)
    
    # 3. *** ALTERAÇÃO PRINCIPAL: Carregar ResNet-152 ***
    base_model = tf.keras.applications.ResNet152(
        weights='imagenet',
        include_top=False,
        input_tensor=x
    )
    
    # 4. Topo personalizado
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    
    x = Dense(150, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
    x = Dropout(0.25)(x)
    
    x = Dense(100, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
    x = Dropout(0.25)(x)
    
    outputs = Dense(2, activation='softmax')(x)
    
    # Compilação (mantido igual)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer='Adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
acc = []
jacc = []
f1 = []
prec = []
rec = []

# Configurar K-Fold com random_state fixo para reprodutibilidade
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=53)

# Verificar quais folds já foram completados
completed_folds = []
for f in range(1, 6):
    model_path = output_dir + f'model_{nome}_fold_{f}.keras'
    if os.path.exists(model_path):
        completed_folds.append(f)
print(f"Folds concluídos: {completed_folds}")

fold_no = 1
histories = []
metrics = []

for train_idx, val_idx in kfold.split(X_train, y_train):
    # Pular folds já concluídos
    if fold_no in completed_folds:
        print(f"\nPulando fold {fold_no} (já concluído)")
        fold_no += 1
        continue

    print(f'\nTreinando Fold {fold_no}/5')
    
    # Split dos dados
    X_train_fold = X_train[train_idx]
    y_train_fold = y_train[train_idx]
    X_val_fold = X_train[val_idx]
    y_val_fold = y_train[val_idx]

    # Pré-processamento final
    X_train_fold = np.expand_dims(X_train_fold, axis=-1)
    X_val_fold = np.expand_dims(X_val_fold, axis=-1)
    y_train_fold_cat = to_categorical(y_train_fold, 2)
    y_val_fold_cat = to_categorical(y_val_fold, 2)

    # Criar novo modelo para cada fold
    model = model_resnet152()

    # Checkpoint com nome do fold
    checkpoint_filepath = output_dir + f'model_{nome}_fold_{fold_no}.keras'
    callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=20, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_filepath,
            save_weights_only=False,
            monitor='val_accuracy',
            mode='max',
            save_best_only=True,
            verbose=1
        )
    ]

    # Calcula tempo (start)
    start_time = time.time()
    
    # Treinar modelo
    history = model.fit(
        X_train_fold, y_train_fold_cat,
        batch_size=BATCH_SIZE,
        epochs=EPOCHS,
        validation_data=(X_val_fold, y_val_fold_cat),
        callbacks=callbacks,
        verbose=1
    )

    # Calcula tempo (end)
    end_time = time.time()

    training_time = end_time - start_time
    print(f"\nO modelo demorou {training_time:.2f} segundos para treinar.")

    # Coletar métricas e salvar modelo
    predictions = model.predict(X_val_fold)
    y_pred = np.argmax(predictions, axis=1)
    
    metrics.append({
        'fold': fold_no,
        'report': classification_report(y_val_fold, y_pred, output_dict=True, zero_division=0),
        'matrix': confusion_matrix(y_val_fold, y_pred)
    })

    # Métricas de classificação (por fold)
    acc.append(accuracy_score(y_val_fold, y_pred))
    jacc.append(jaccard_score(y_val_fold, y_pred))
    f1.append(f1_score(y_val_fold, y_pred))
    prec.append(precision_score(y_val_fold, y_pred))
    rec.append(recall_score(y_val_fold, y_pred))

    # Salvar métricas em um arquivo .txt
    metrics_filename = os.path.join(output_dir, f'metrics_{nome}_fold_{fold_no}.txt')
    with open(metrics_filename, 'w') as f:
        f.write(f"Fold {fold_no} Metrics:\n")
        f.write(f"Accuracy: {acc[-1]}\n")
        f.write(f"Jaccard Score: {jacc[-1]}\n")
        f.write(f"F1 Score: {f1[-1]}\n")
        f.write(f"Precision: {prec[-1]}\n")
        f.write(f"Recall: {rec[-1]}\n")
        f.write("\nClassification Report:\n")
        f.write(classification_report(y_val_fold, y_pred, zero_division=0))
        f.write("\nConfusion Matrix:\n")
        f.write(np.array2string(confusion_matrix(y_val_fold, y_pred)))

    # Limpeza de memória
    del model
    tf.keras.backend.clear_session()
    gc.collect()

    # Salvar checkpoint a cada 3 folds
    if fold_no % 3 == 0:
        print(f"\nCheckpoint: Folds {fold_no-2}-{fold_no} concluídos")

    fold_no += 1

Folds concluídos: []

Treinando Fold 1/5


I0000 00:00:1744046309.959062    1891 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744046309.959173    1891 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744046309.959228    1891 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744046312.924465    1891 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1744046312.924571    1891 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:10:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-04-07

Epoch 1/200


I0000 00:00:1744046379.059145    1969 service.cc:146] XLA service 0x7f71a8001d20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744046379.059242    1969 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3060, Compute Capability 8.6
2025-04-07 14:19:41.974127: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-04-07 14:19:48.743953: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 90101
2025-04-07 14:19:59.816059: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_45152', 196 bytes spill stores, 196 bytes spill loads

2025-04-07 14:20:43.444591: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in fu

40/41 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step - accuracy: 0.7728 - loss: 3.9035

2025-04-07 14:21:53.597301: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'loop_add_divide_multiply_subtract_fusion_159', 4 bytes spill stores, 16 bytes spill loads



41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.7753 - loss: 3.8846   

2025-04-07 14:22:10.454555: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4961', 8 bytes spill stores, 8 bytes spill loads

2025-04-07 14:22:11.121526: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4961', 8 bytes spill stores, 8 bytes spill loads




Epoch 1: val_accuracy improved from -inf to 0.50000, saving model to 13-CNN-espectro-ResNet152 results (2%)/model_Yildirim_fold_1.keras
41/41 ━━━━━━━━━━━━━━━━━━━━ 266s 4s/step - accuracy: 0.7776 - loss: 3.8667 - val_accuracy: 0.5000 - val_loss: 43837591552.0000
Epoch 2/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - accuracy: 0.9761 - loss: 1.4263
Epoch 2: val_accuracy did not improve from 0.50000
41/41 ━━━━━━━━━━━━━━━━━━━━ 19s 446ms/step - accuracy: 0.9762 - loss: 1.4204 - val_accuracy: 0.5000 - val_loss: 2175.7332
Epoch 3/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 347ms/step - accuracy: 0.9992 - loss: 0.6077
Epoch 3: val_accuracy improved from 0.50000 to 0.51235, saving model to 13-CNN-espectro-ResNet152 results (2%)/model_Yildirim_fold_1.keras
41/41 ━━━━━━━━━━━━━━━━━━━━ 50s 1s/step - accuracy: 0.9991 - loss: 0.6059 - val_accuracy: 0.5123 - val_loss: 1.0687
Epoch 4/200
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 421ms/step - accuracy: 0.9958 - loss: 0.3294
Epoch 4: val_accuracy did not improve from 0.51235

In [ ]:
print("Accuracy: "+ str(np.mean(acc)) + "+- " + str(np.std(acc)))
print("Jaccard: "+ str(np.mean(jacc)) + "+- " + str(np.std(jacc)))
print("Dice: "+ str(np.mean(f1)) + "+- " + str(np.std(f1)))
print("Precision: "+ str(np.mean(prec)) + "+- " + str(np.std(prec)))
print("Recall: "+ str(np.mean(rec)) + "+- " + str(np.std(rec)))

In [ ]:
# GC collect
tf.keras.backend.clear_session()
gc.collect()